In [35]:
accs =  {0: 0.48, 1: 0.12, 2: 0.0, 3: 0.04, 4: 0.0, 5: 0.08, 6: 0.0, 7: 0.02, 8: 0.14, 9: 0.5, 10: 0.22, 11: 0.34, 12: 0.42, 13: 0.42, 14: 0.4, 15: 0.36, 16: 0.26, 17: 0.34, 18: 0.14, 19: 0.28, 20: 0.44, 21: 0.1, 22: 0.4, 23: 0.16, 24: 0.72, 25: 0.46, 26: 0.06, 27: 0.22, 28: 0.28, 29: 0.3, 30: 0.18, 31: 0.2, 32: 0.06, 33: 0.0, 34: 0.06, 35: 0.0, 36: 0.0, 37: 0.2, 38: 0.16, 39: 0.32, 40: 0.0, 41: 0.2, 42: 0.24, 43: 0.08, 44: 0.1, 45: 0.32, 46: 0.0, 47: 0.36, 48: 0.24, 49: 0.08, 50: 0.02, 51: 0.24, 52: 0.12, 53: 0.14, 54: 0.02, 55: 0.08, 56: 0.36, 57: 0.34, 58: 0.0, 59: 0.06, 60: 0.0, 61: 0.06, 62: 0.0, 63: 0.2, 64: 0.1, 65: 0.1, 66: 0.06, 67: 0.02, 68: 0.0, 69: 0.24, 70: 0.46, 71: 0.48, 72: 0.22, 73: 0.06, 74: 0.0, 75: 0.36, 76: 0.18, 77: 0.0, 78: 0.24, 79: 0.26, 80: 0.12, 81: 0.2, 82: 0.16, 83: 0.56, 84: 0.44, 85: 0.36, 86: 0.22, 87: 0.5, 88: 0.58, 89: 0.16, 90: 0.66, 91: 0.46, 92: 0.46, 93: 0.28, 94: 0.42, 95: 0.7, 96: 0.4, 97: 0.4, 98: 0.54, 99: 0.2, 100: 0.52, 101: 0.02, 102: 0.64, 103: 0.06, 104: 0.18, 105: 0.2, 106: 0.16, 107: 0.22, 108: 0.08, 109: 0.26, 110: 0.2, 111: 0.22, 112: 0.0, 113: 0.14, 114: 0.1, 115: 0.08, 116: 0.1, 117: 0.32, 118: 0.02, 119: 0.1, 120: 0.18, 121: 0.22, 122: 0.02, 123: 0.16, 124: 0.0, 125: 0.1, 126: 0.08, 127: 0.34, 128: 0.14, 129: 0.38, 130: 0.58, 131: 0.2, 132: 0.12, 133: 0.5, 134: 0.16, 135: 0.68, 136: 0.48, 137: 0.34, 138: 0.42, 139: 0.72, 140: 0.28, 141: 0.58, 142: 0.38, 143: 0.48, 144: 0.34, 145: 0.56, 146: 0.12, 147: 0.12, 148: 0.16, 149: 0.16, 150: 0.04, 151: 0.0, 152: 0.06, 153: 0.0, 154: 0.06, 155: 0.0, 156: 0.1, 157: 0.22, 158: 0.0, 159: 0.0, 160: 0.14, 161: 0.0, 162: 0.0, 163: 0.04, 164: 0.1, 165: 0.0, 166: 0.0, 167: 0.0, 168: 0.0, 169: 0.06, 170: 0.0, 171: 0.0, 172: 0.02, 173: 0.02, 174: 0.12, 175: 0.04, 176: 0.02, 177: 0.0, 178: 0.14, 179: 0.0, 180: 0.0, 181: 0.16, 182: 0.04, 183: 0.04, 184: 0.0, 185: 0.0, 186: 0.0, 187: 0.0, 188: 0.0, 189: 0.0, 190: 0.02, 191: 0.02, 192: 0.04, 193: 0.0, 194: 0.06, 195: 0.0, 196: 0.0, 197: 0.0, 198: 0.0, 199: 0.1, 200: 0.0, 201: 0.0, 202: 0.0, 203: 0.08, 204: 0.0, 205: 0.06, 206: 0.02, 207: 0.0, 208: 0.0, 209: 0.0, 210: 0.16, 211: 0.0, 212: 0.02, 213: 0.06, 214: 0.06, 215: 0.04, 216: 0.12, 217: 0.02, 218: 0.0, 219: 0.06, 220: 0.06, 221: 0.06, 222: 0.0, 223: 0.04, 224: 0.06, 225: 0.0, 226: 0.0, 227: 0.0, 228: 0.16, 229: 0.08, 230: 0.0, 231: 0.0, 232: 0.02, 233: 0.0, 234: 0.02, 235: 0.06, 236: 0.0, 237: 0.0, 238: 0.0, 239: 0.0, 240: 0.0, 241: 0.0, 242: 0.04, 243: 0.02, 244: 0.18, 245: 0.06, 246: 0.0, 247: 0.2, 248: 0.0, 249: 0.02, 250: 0.0, 251: 0.16, 252: 0.02, 253: 0.08, 254: 0.04, 255: 0.08, 256: 0.0, 257: 0.02, 258: 0.16, 259: 0.16, 260: 0.14, 261: 0.12, 262: 0.02, 263: 0.04, 264: 0.0, 265: 0.0, 266: 0.0, 267: 0.0, 268: 0.02, 269: 0.0, 270: 0.04, 271: 0.14, 272: 0.02, 273: 0.02, 274: 0.42, 275: 0.62, 276: 0.38, 277: 0.0, 278: 0.0, 279: 0.2, 280: 0.04, 281: 0.0, 282: 0.0, 283: 0.1, 284: 0.18, 285: 0.0, 286: 0.24, 287: 0.16, 288: 0.1, 289: 0.2, 290: 0.14, 291: 0.44, 292: 0.16, 293: 0.36, 294: 0.26, 295: 0.08, 296: 0.14, 297: 0.04, 298: 0.32, 299: 0.04, 300: 0.56, 301: 0.08, 302: 0.12, 303: 0.06, 304: 0.02, 305: 0.16, 306: 0.26, 307: 0.32, 308: 0.26, 309: 0.1, 310: 0.08, 311: 0.02, 312: 0.0, 313: 0.02, 314: 0.06, 315: 0.1, 316: 0.26, 317: 0.44, 318: 0.34, 319: 0.1, 320: 0.16, 321: 0.52, 322: 0.56, 323: 0.62, 324: 0.26, 325: 0.54, 326: 0.36, 327: 0.18, 328: 0.18, 329: 0.08, 330: 0.14, 331: 0.3, 332: 0.04, 333: 0.06, 334: 0.16, 335: 0.14, 336: 0.14, 337: 0.22, 338: 0.28, 339: 0.22, 340: 0.74, 341: 0.02, 342: 0.08, 343: 0.1, 344: 0.18, 345: 0.0, 346: 0.02, 347: 0.26, 348: 0.0, 349: 0.0, 350: 0.2, 351: 0.42, 352: 0.04, 353: 0.08, 354: 0.18, 355: 0.02, 356: 0.0, 357: 0.0, 358: 0.0, 359: 0.02, 360: 0.02, 361: 0.26, 362: 0.36, 363: 0.46, 364: 0.28, 365: 0.1, 366: 0.16, 367: 0.1, 368: 0.2, 369: 0.1, 370: 0.14, 371: 0.2, 372: 0.08, 373: 0.0, 374: 0.06, 375: 0.12, 376: 0.5, 377: 0.02, 378: 0.1, 379: 0.02, 380: 0.0, 381: 0.04, 382: 0.08, 383: 0.08, 384: 0.3, 385: 0.0, 386: 0.0, 387: 0.62, 388: 0.34, 389: 0.08, 390: 0.04, 391: 0.06, 392: 0.38, 393: 0.32, 394: 0.24, 395: 0.04, 396: 0.5, 397: 0.12, 398: 0.4, 399: 0.0, 400: 0.0, 401: 0.1, 402: 0.08, 403: 0.06, 404: 0.3, 405: 0.18, 406: 0.02, 407: 0.38, 408: 0.02, 409: 0.0, 410: 0.08, 411: 0.02, 412: 0.0, 413: 0.0, 414: 0.0, 415: 0.0, 416: 0.0, 417: 0.42, 418: 0.0, 419: 0.0, 420: 0.06, 421: 0.02, 422: 0.0, 423: 0.0, 424: 0.0, 425: 0.16, 426: 0.1, 427: 0.04, 428: 0.0, 429: 0.34, 430: 0.12, 431: 0.0, 432: 0.06, 433: 0.0, 434: 0.0, 435: 0.0, 436: 0.0, 437: 0.14, 438: 0.08, 439: 0.08, 440: 0.02, 441: 0.06, 442: 0.02, 443: 0.12, 444: 0.04, 445: 0.0, 446: 0.0, 447: 0.0, 448: 0.0, 449: 0.02, 450: 0.18, 451: 0.16, 452: 0.0, 453: 0.02, 454: 0.02, 455: 0.32, 456: 0.0, 457: 0.02, 458: 0.3, 459: 0.02, 460: 0.0, 461: 0.0, 462: 0.06, 463: 0.04, 464: 0.0, 465: 0.0, 466: 0.36, 467: 0.1, 468: 0.1, 469: 0.0, 470: 0.0, 471: 0.02, 472: 0.02, 473: 0.0, 474: 0.04, 475: 0.14, 476: 0.18, 477: 0.04, 478: 0.0, 479: 0.04, 480: 0.02, 481: 0.08, 482: 0.0, 483: 0.04, 484: 0.0, 485: 0.0, 486: 0.12, 487: 0.0, 488: 0.02, 489: 0.0, 490: 0.18, 491: 0.1, 492: 0.0, 493: 0.04, 494: 0.0, 495: 0.1, 496: 0.1, 497: 0.02, 498: 0.06, 499: 0.0, 500: 0.22, 501: 0.0, 502: 0.0, 503: 0.0, 504: 0.0, 505: 0.0, 506: 0.06, 507: 0.12, 508: 0.14, 509: 0.0, 510: 0.18, 511: 0.0, 512: 0.04, 513: 0.0, 514: 0.1, 515: 0.0, 516: 0.0, 517: 0.16, 518: 0.08, 519: 0.04, 520: 0.04, 521: 0.0, 522: 0.04, 523: 0.0, 524: 0.0, 525: 0.04, 526: 0.0, 527: 0.0, 528: 0.32, 529: 0.0, 530: 0.04, 531: 0.0, 532: 0.04, 533: 0.4, 534: 0.0, 535: 0.14, 536: 0.02, 537: 0.18, 538: 0.14, 539: 0.0, 540: 0.16, 541: 0.0, 542: 0.02, 543: 0.0, 544: 0.0, 545: 0.2, 546: 0.02, 547: 0.2, 548: 0.28, 549: 0.0, 550: 0.02, 551: 0.0, 552: 0.04, 553: 0.12, 554: 0.1, 555: 0.14, 556: 0.02, 557: 0.0, 558: 0.0, 559: 0.0, 560: 0.08, 561: 0.06, 562: 0.04, 563: 0.1, 564: 0.16, 565: 0.1, 566: 0.24, 567: 0.0, 568: 0.0, 569: 0.08, 570: 0.02, 571: 0.04, 572: 0.06, 573: 0.3, 574: 0.38, 575: 0.06, 576: 0.38, 577: 0.0, 578: 0.0, 579: 0.08, 580: 0.04, 581: 0.02, 582: 0.0, 583: 0.08, 584: 0.0, 585: 0.0, 586: 0.16, 587: 0.0, 588: 0.0, 589: 0.04, 590: 0.0, 591: 0.0, 592: 0.2, 593: 0.04, 594: 0.26, 595: 0.02, 596: 0.0, 597: 0.04, 598: 0.02, 599: 0.16, 600: 0.0, 601: 0.0, 602: 0.0, 603: 0.04, 604: 0.2, 605: 0.2, 606: 0.0, 607: 0.54, 608: 0.02, 609: 0.12, 610: 0.04, 611: 0.28, 612: 0.08, 613: 0.06, 614: 0.0, 615: 0.1, 616: 0.08, 617: 0.0, 618: 0.02, 619: 0.0, 620: 0.0, 621: 0.0, 622: 0.04, 623: 0.0, 624: 0.0, 625: 0.5, 626: 0.0, 627: 0.08, 628: 0.32, 629: 0.14, 630: 0.02, 631: 0.0, 632: 0.0, 633: 0.0, 634: 0.0, 635: 0.02, 636: 0.0, 637: 0.14, 638: 0.0, 639: 0.0, 640: 0.5, 641: 0.06, 642: 0.04, 643: 0.0, 644: 0.04, 645: 0.18, 646: 0.12, 647: 0.1, 648: 0.0, 649: 0.12, 650: 0.02, 651: 0.0, 652: 0.0, 653: 0.1, 654: 0.0, 655: 0.0, 656: 0.0, 657: 0.0, 658: 0.16, 659: 0.28, 660: 0.0, 661: 0.18, 662: 0.0, 663: 0.0, 664: 0.0, 665: 0.02, 666: 0.0, 667: 0.0, 668: 0.28, 669: 0.04, 670: 0.0, 671: 0.0, 672: 0.08, 673: 0.0, 674: 0.02, 675: 0.0, 676: 0.0, 677: 0.0, 678: 0.0, 679: 0.1, 680: 0.02, 681: 0.02, 682: 0.36, 683: 0.06, 684: 0.1, 685: 0.66, 686: 0.04, 687: 0.32, 688: 0.24, 689: 0.0, 690: 0.02, 691: 0.0, 692: 0.0, 693: 0.0, 694: 0.3, 695: 0.04, 696: 0.02, 697: 0.0, 698: 0.06, 699: 0.18, 700: 0.0, 701: 0.38, 702: 0.0, 703: 0.06, 704: 0.1, 705: 0.02, 706: 0.0, 707: 0.02, 708: 0.0, 709: 0.04, 710: 0.0, 711: 0.0, 712: 0.02, 713: 0.06, 714: 0.2, 715: 0.14, 716: 0.04, 717: 0.02, 718: 0.0, 719: 0.18, 720: 0.02, 721: 0.0, 722: 0.04, 723: 0.18, 724: 0.14, 725: 0.0, 726: 0.06, 727: 0.32, 728: 0.0, 729: 0.0, 730: 0.0, 731: 0.0, 732: 0.04, 733: 0.0, 734: 0.08, 735: 0.02, 736: 0.18, 737: 0.04, 738: 0.0, 739: 0.08, 740: 0.0, 741: 0.24, 742: 0.0, 743: 0.02, 744: 0.0, 745: 0.08, 746: 0.38, 747: 0.02, 748: 0.0, 749: 0.02, 750: 0.0, 751: 0.0, 752: 0.08, 753: 0.06, 754: 0.0, 755: 0.28, 756: 0.02, 757: 0.16, 758: 0.0, 759: 0.06, 760: 0.0, 761: 0.12, 762: 0.0, 763: 0.2, 764: 0.0, 765: 0.06, 766: 0.16, 767: 0.04, 768: 0.1, 769: 0.04, 770: 0.18, 771: 0.06, 772: 0.02, 773: 0.08, 774: 0.0, 775: 0.0, 776: 0.14, 777: 0.0, 778: 0.04, 779: 0.46, 780: 0.06, 781: 0.12, 782: 0.0, 783: 0.3, 784: 0.0, 785: 0.0, 786: 0.0, 787: 0.0, 788: 0.0, 789: 0.06, 790: 0.02, 791: 0.0, 792: 0.0, 793: 0.06, 794: 0.02, 795: 0.0, 796: 0.12, 797: 0.0, 798: 0.26, 799: 0.0, 800: 0.3, 801: 0.02, 802: 0.12, 803: 0.04, 804: 0.0, 805: 0.32, 806: 0.02, 807: 0.0, 808: 0.02, 809: 0.0, 810: 0.0, 811: 0.0, 812: 0.2, 813: 0.0, 814: 0.08, 815: 0.18, 816: 0.14, 817: 0.02, 818: 0.0, 819: 0.0, 820: 0.24, 821: 0.08, 822: 0.06, 823: 0.0, 824: 0.0, 825: 0.08, 826: 0.08, 827: 0.0, 828: 0.04, 829: 0.14, 830: 0.0, 831: 0.02, 832: 0.22, 833: 0.06, 834: 0.02, 835: 0.04, 836: 0.0, 837: 0.0, 838: 0.0, 839: 0.04, 840: 0.02, 841: 0.0, 842: 0.0, 843: 0.02, 844: 0.0, 845: 0.06, 846: 0.0, 847: 0.02, 848: 0.0, 849: 0.02, 850: 0.02, 851: 0.04, 852: 0.14, 853: 0.0, 854: 0.08, 855: 0.18, 856: 0.0, 857: 0.0, 858: 0.08, 859: 0.0, 860: 0.0, 861: 0.0, 862: 0.0, 863: 0.06, 864: 0.0, 865: 0.0, 866: 0.0, 867: 0.12, 868: 0.0, 869: 0.0, 870: 0.02, 871: 0.0, 872: 0.02, 873: 0.48, 874: 0.46, 875: 0.04, 876: 0.0, 877: 0.02, 878: 0.12, 879: 0.02, 880: 0.0, 881: 0.1, 882: 0.0, 883: 0.0, 884: 0.06, 885: 0.0, 886: 0.2, 887: 0.02, 888: 0.32, 889: 0.02, 890: 0.04, 891: 0.16, 892: 0.06, 893: 0.0, 894: 0.0, 895: 0.04, 896: 0.0, 897: 0.14, 898: 0.0, 899: 0.0, 900: 0.54, 901: 0.08, 902: 0.0, 903: 0.02, 904: 0.12, 905: 0.02, 906: 0.0, 907: 0.0, 908: 0.08, 909: 0.0, 910: 0.0, 911: 0.04, 912: 0.04, 913: 0.04, 914: 0.12, 915: 0.08, 916: 0.18, 917: 0.06, 918: 0.48, 919: 0.08, 920: 0.16, 921: 0.0, 922: 0.22, 923: 0.0, 924: 0.06, 925: 0.06, 926: 0.0, 927: 0.26, 928: 0.0, 929: 0.0, 930: 0.0, 931: 0.08, 932: 0.2, 933: 0.04, 934: 0.04, 935: 0.0, 936: 0.12, 937: 0.16, 938: 0.32, 939: 0.04, 940: 0.02, 941: 0.12, 942: 0.06, 943: 0.0, 944: 0.14, 945: 0.26, 946: 0.26, 947: 0.02, 948: 0.22, 949: 0.1, 950: 0.08, 951: 0.1, 952: 0.2, 953: 0.3, 954: 0.06, 955: 0.34, 956: 0.24, 957: 0.38, 958: 0.12, 959: 0.38, 960: 0.0, 961: 0.0, 962: 0.02, 963: 0.16, 964: 0.04, 965: 0.0, 966: 0.06, 967: 0.0, 968: 0.0, 969: 0.0, 970: 0.0, 971: 0.02, 972: 0.0, 973: 0.0, 974: 0.08, 975: 0.0, 976: 0.0, 977: 0.0, 978: 0.0, 979: 0.0, 980: 0.0, 981: 0.0, 982: 0.0, 983: 0.04, 984: 0.4, 985: 0.44, 986: 0.8, 987: 0.02, 988: 0.2, 989: 0.38, 990: 0.18, 991: 0.12, 992: 0.24, 993: 0.48, 994: 0.34, 995: 0.36, 996: 0.02, 997: 0.04, 998: 0.0, 999: 0.0}

In [36]:
import numpy as np

np.mean(list(accs.values()))

0.10772